<a href="https://colab.research.google.com/github/alfredwisana/big-data_project/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('local[*]').appName('myApp').getOrCreate()
import matplotlib.pyplot as plt
import seaborn

In [2]:
# Read the CSV file containing Titanic dataset into Spark's DataFrame and show it

df = spark.read.format('csv').options(header=True, inferSchema=True).load('Clean_Dataset.csv')
df.show(30,truncate=50)

+---+---------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|_c0|  airline| flight|source_city|departure_time|stops| arrival_time|destination_city|  class|duration|days_left|price|
+---+---------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|  0| SpiceJet|SG-8709|      Delhi|       Evening| zero|        Night|          Mumbai|Economy|    2.17|        1| 5953|
|  1| SpiceJet|SG-8157|      Delhi| Early_Morning| zero|      Morning|          Mumbai|Economy|    2.33|        1| 5953|
|  2|  AirAsia| I5-764|      Delhi| Early_Morning| zero|Early_Morning|          Mumbai|Economy|    2.17|        1| 5956|
|  3|  Vistara| UK-995|      Delhi|       Morning| zero|    Afternoon|          Mumbai|Economy|    2.25|        1| 5955|
|  4|  Vistara| UK-963|      Delhi|       Morning| zero|      Morning|          Mumbai|Economy|    2.33|        1| 5955|
|  5|  Vistara| UK-945|      Del

In [3]:
# Try to print the schema

df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- airline: string (nullable = true)
 |-- flight: string (nullable = true)
 |-- source_city: string (nullable = true)
 |-- departure_time: string (nullable = true)
 |-- stops: string (nullable = true)
 |-- arrival_time: string (nullable = true)
 |-- destination_city: string (nullable = true)
 |-- class: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- days_left: integer (nullable = true)
 |-- price: integer (nullable = true)



In [4]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [5]:
df.select([count(when(isnan(c) | isnull(c),1)).alias(c) for c in df.columns]).show()

+---+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|_c0|airline|flight|source_city|departure_time|stops|arrival_time|destination_city|class|duration|days_left|price|
+---+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+
|  0|      0|     0|          0|             0|    0|           0|               0|    0|       0|        0|    0|
+---+-------+------+-----------+--------------+-----+------------+----------------+-----+--------+---------+-----+



<h1>Data Preprocessing

In [6]:
from pyspark.ml.feature import Normalizer,VectorAssembler, StringIndexer
from pyspark.ml import Pipeline

In [7]:
from pyspark.sql.functions import col, max as spark_max, abs as spark_abs

max_abs_value = df.select(spark_max(spark_abs(col("price"))).alias("max_abs_value")).collect()[0]["max_abs_value"]


In [8]:
print(max_abs_value)
len(str(int(max_abs_value)))

123071


6

In [9]:
scaling_factor = 10 ** (len(str(int(max_abs_value)))-1)
df = df.withColumn("normalized_price", col("price") / scaling_factor)
df.show(30,truncate=50)

+---+---------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+----------------+
|_c0|  airline| flight|source_city|departure_time|stops| arrival_time|destination_city|  class|duration|days_left|price|normalized_price|
+---+---------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+----------------+
|  0| SpiceJet|SG-8709|      Delhi|       Evening| zero|        Night|          Mumbai|Economy|    2.17|        1| 5953|         0.05953|
|  1| SpiceJet|SG-8157|      Delhi| Early_Morning| zero|      Morning|          Mumbai|Economy|    2.33|        1| 5953|         0.05953|
|  2|  AirAsia| I5-764|      Delhi| Early_Morning| zero|Early_Morning|          Mumbai|Economy|    2.17|        1| 5956|         0.05956|
|  3|  Vistara| UK-995|      Delhi|       Morning| zero|    Afternoon|          Mumbai|Economy|    2.25|        1| 5955|         0.05955|
|  4|  Vistara| UK-963|      Delhi

In [10]:
column_name = df.columns

In [11]:
column_name.remove('_c0')
column_name.remove('price')
column_name.remove('flight')
print(column_name)

['airline', 'source_city', 'departure_time', 'stops', 'arrival_time', 'destination_city', 'class', 'duration', 'days_left', 'normalized_price']


In [12]:
final_df = df.select(column_name)


indexers = [StringIndexer(inputCol=col, outputCol=col+'_idx').fit(final_df) for col in ['airline','source_city','departure_time', 'stops', 'arrival_time', 'destination_city','class']]
pipeline = Pipeline(stages=indexers)
final_df = pipeline.fit(final_df).transform(final_df)
final_df.show()

+---------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+----------------+-----------+---------------+------------------+---------+----------------+--------------------+---------+
|  airline|source_city|departure_time|stops| arrival_time|destination_city|  class|duration|days_left|normalized_price|airline_idx|source_city_idx|departure_time_idx|stops_idx|arrival_time_idx|destination_city_idx|class_idx|
+---------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+----------------+-----------+---------------+------------------+---------+----------------+--------------------+---------+
| SpiceJet|      Delhi|       Evening| zero|        Night|          Mumbai|Economy|    2.17|        1|         0.05953|        5.0|            0.0|               2.0|      1.0|             0.0|                 0.0|      0.0|
| SpiceJet|      Delhi| Early_Morning| zero|      Morning|          Mumbai|Economy|    2.33|        

In [13]:
assembler = VectorAssembler(inputCols=['airline_idx', 'source_city_idx', 'departure_time_idx', 'stops_idx', 'arrival_time_idx', 'destination_city_idx', 'class_idx','duration', 'days_left'], outputCol="features")
final_df = assembler.transform(final_df)
final_df.show(30,truncate=50)

+---------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+----------------+-----------+---------------+------------------+---------+----------------+--------------------+---------+---------------------------------------+
|  airline|source_city|departure_time|stops| arrival_time|destination_city|  class|duration|days_left|normalized_price|airline_idx|source_city_idx|departure_time_idx|stops_idx|arrival_time_idx|destination_city_idx|class_idx|                               features|
+---------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+----------------+-----------+---------------+------------------+---------+----------------+--------------------+---------+---------------------------------------+
| SpiceJet|      Delhi|       Evening| zero|        Night|          Mumbai|Economy|    2.17|        1|         0.05953|        5.0|            0.0|               2.0|      1.0|             0.0|            

In [14]:
# Initialize the StandardScaler
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withMean=True, withStd=True)

# Fit the scaler to the data
scaler_model = scaler.fit(final_df)

# Transform the data
final_df = scaler_model.transform(final_df)

final_df.show(10,truncate=10)

+--------+-----------+--------------+-----+------------+----------------+-------+--------+---------+----------------+-----------+---------------+------------------+---------+----------------+--------------------+---------+----------+--------------+
| airline|source_city|departure_time|stops|arrival_time|destination_city|  class|duration|days_left|normalized_price|airline_idx|source_city_idx|departure_time_idx|stops_idx|arrival_time_idx|destination_city_idx|class_idx|  features|scaledFeatures|
+--------+-----------+--------------+-----+------------+----------------+-------+--------+---------+----------------+-----------+---------------+------------------+---------+----------------+--------------------+---------+----------+--------------+
|SpiceJet|      Delhi|       Evening| zero|       Night|          Mumbai|Economy|    2.17|        1|         0.05953|        5.0|            0.0|               2.0|      1.0|             0.0|                 0.0|      0.0|[5.0,0....|    [2.8530...|
|Spi

In [15]:
from pyspark.ml.feature import PCA


pca = PCA(k=2, inputCol="scaledFeatures", outputCol="pcafeatures")
model = pca.fit(final_df)

# Step 5: Transform Data
final_df = model.transform(final_df)


In [16]:
final_df.show(10, truncate=50)

+--------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+----------------+-----------+---------------+------------------+---------+----------------+--------------------+---------+--------------------------------------+--------------------------------------------------+-------------------------------------------+
| airline|source_city|departure_time|stops| arrival_time|destination_city|  class|duration|days_left|normalized_price|airline_idx|source_city_idx|departure_time_idx|stops_idx|arrival_time_idx|destination_city_idx|class_idx|                              features|                                    scaledFeatures|                                pcafeatures|
+--------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+----------------+-----------+---------------+------------------+---------+----------------+--------------------+---------+--------------------------------------+----------------------

In [17]:
feature_vector_df = final_df.select(['features','pcafeatures', 'normalized_price'])
feature_vector_df.show(truncate=False)

+---------------------------------------+-------------------------------------------+----------------+
|features                               |pcafeatures                                |normalized_price|
+---------------------------------------+-------------------------------------------+----------------+
|[5.0,0.0,2.0,1.0,0.0,0.0,0.0,2.17,1.0] |[-3.1081011449371894,-0.13769552216662556] |0.05953         |
|[5.0,0.0,1.0,1.0,2.0,0.0,0.0,2.33,1.0] |[-3.3858235710522706,-0.05359819764615822] |0.05953         |
|[4.0,0.0,1.0,1.0,4.0,0.0,0.0,2.17,1.0] |[-3.291962039575339,-0.04781304747503553]  |0.05956         |
|(9,[3,4,7,8],[1.0,3.0,2.25,1.0])       |[-1.3797307554263523,-0.048865723287855135]|0.05955         |
|(9,[3,4,7,8],[1.0,2.0,2.33,1.0])       |[-1.2129029513930916,-0.059042951387626566]|0.05955         |
|(9,[3,4,7,8],[1.0,3.0,2.33,1.0])       |[-1.374581190423971,-0.049780450307782186] |0.05955         |
|(9,[3,4,7,8],[1.0,2.0,2.08,1.0])       |[-1.2289953420255326,-0.05618442

<h3> Regression Model

In [18]:
from pyspark.ml.evaluation import RegressionEvaluator

In [19]:
(trainData, testData) = feature_vector_df.randomSplit([0.8,0.2],seed = 2)

<h5> XGBoost

In [20]:
from pyspark.ml.regression import GBTRegressor
import xgboost as xg
import sklearn
import numpy as np

In [21]:


xgb_r = xg.XGBRegressor(objective ='reg:linear',n_estimators = 10, seed = 123).fit(np.vstack(trainData.select("pcafeatures").collect()), np.vstack(trainData.select("normalized_price").collect()))

c:\Users\Alfred\anaconda3\envs\kuliah\Lib\site-packages\xgboost\core.py:160: UserWarning: [22:11:51] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [22]:
pred = xgb_r.predict(np.vstack(testData.select("pcafeatures").collect()))
pred

array([0.13222344, 0.13222344, 0.13222344, ..., 0.3660522 , 0.2886305 ,
       0.23621905], dtype=float32)

In [23]:
from pyspark.sql.functions import monotonically_increasing_id

test_pred = spark.createDataFrame(pred)

testData_pred = testData.withColumn('key', monotonically_increasing_id())
test_pred = test_pred.withColumn('key', monotonically_increasing_id())

test_pred = test_pred.withColumn("prediction", test_pred["value"])
# Perform the join on the key column
test_with_pred = testData_pred.join(test_pred, on='key')


test_with_pred = test_with_pred.drop('key')

test_with_pred.show(10, truncate=50)

+----------------------------------+-------------------------------------------+----------------+----------+----------+
|                          features|                                pcafeatures|normalized_price|     value|prediction|
+----------------------------------+-------------------------------------------+----------------+----------+----------+
| (9,[0,2,7,8],[1.0,1.0,13.25,6.0])| [0.013102157639334477,0.08219716317632633]|         0.14775|0.13222344|0.13222344|
|(9,[0,2,7,8],[1.0,1.0,13.25,16.0])|[-0.024231767643273568,0.12517708677420586]|         0.07262|0.13222344|0.13222344|
|(9,[0,2,7,8],[1.0,1.0,13.25,28.0])| [-0.06903247798240324,0.17675299509166134]|         0.05281|0.13222344|0.13222344|
|(9,[0,2,7,8],[1.0,1.0,13.25,31.0])|  [-0.08023265556718565,0.1896469721710252]|         0.06231|0.13222344|0.13222344|
|(9,[0,2,7,8],[1.0,1.0,13.25,34.0])| [-0.09143283315196807,0.20254094925038907]|         0.06231|0.13222344|0.13222344|
|(9,[0,2,7,8],[1.0,1.0,13.25,37.0])| [-0

In [24]:
evaluator = RegressionEvaluator(labelCol='normalized_price', predictionCol='prediction', metricName='rmse')
rmse = evaluator.evaluate(test_with_pred)
print ("Root Mean Square of XGBoost model is %g"%rmse)

Root Mean Square of XGBoost model is 0.279822


<h5> Random Forest Regressor

In [25]:
from pyspark.ml.regression import RandomForestRegressor

rf = RandomForestRegressor(featuresCol='pcafeatures',labelCol='normalized_price').fit(trainData)
prediction = rf.transform(testData)

In [26]:
prediction.show(10, truncate=50)

+----------------------------------+-------------------------------------------+----------------+-------------------+
|                          features|                                pcafeatures|normalized_price|         prediction|
+----------------------------------+-------------------------------------------+----------------+-------------------+
| (9,[0,2,7,8],[1.0,1.0,13.25,6.0])| [0.013102157639334477,0.08219716317632633]|         0.14775|0.11531371144470706|
|(9,[0,2,7,8],[1.0,1.0,13.25,16.0])|[-0.024231767643273568,0.12517708677420586]|         0.07262|0.10370166144439515|
|(9,[0,2,7,8],[1.0,1.0,13.25,28.0])| [-0.06903247798240324,0.17675299509166134]|         0.05281|0.10370166144439515|
|(9,[0,2,7,8],[1.0,1.0,13.25,31.0])|  [-0.08023265556718565,0.1896469721710252]|         0.06231|0.10370166144439515|
|(9,[0,2,7,8],[1.0,1.0,13.25,34.0])| [-0.09143283315196807,0.20254094925038907]|         0.06231|0.10370166144439515|
|(9,[0,2,7,8],[1.0,1.0,13.25,37.0])| [-0.102633010736750

In [27]:
evaluator = RegressionEvaluator(labelCol='normalized_price', predictionCol='prediction', metricName='rmse')
rmse = evaluator.evaluate(prediction)
print ("Root Mean Square of Random Forest model is %g"%rmse)

Root Mean Square of Random Forest model is 0.131326


<h5> Support Vector Regressor

In [31]:
from sklearn.svm import LinearSVR

In [32]:
svr = LinearSVR(random_state=0,tol=1e-5,epsilon=0.1,dual=True)

In [33]:
svr_model = svr.fit(np.vstack(trainData.select("pcafeatures").collect()),np.vstack(trainData.select("normalized_price").collect()))

c:\Users\Alfred\anaconda3\envs\kuliah\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Alfred\anaconda3\envs\kuliah\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [34]:
y_pred = svr.predict(np.vstack(testData.select("pcafeatures").collect()))

In [35]:
from pyspark.sql.functions import monotonically_increasing_id

test_pred = spark.createDataFrame(y_pred)

testData_pred = testData.withColumn('key', monotonically_increasing_id())
test_pred = test_pred.withColumn('key', monotonically_increasing_id())

test_pred = test_pred.withColumn("prediction", test_pred["value"])
# Perform the join on the key column
test_with_pred = testData_pred.join(test_pred, on='key')


test_with_pred = test_with_pred.drop('key')
test_with_pred = test_with_pred.drop('value')

test_with_pred.show(10, truncate=50)

+----------------------------------+-------------------------------------------+----------------+-------------------+
|                          features|                                pcafeatures|normalized_price|         prediction|
+----------------------------------+-------------------------------------------+----------------+-------------------+
| (9,[0,2,7,8],[1.0,1.0,13.25,6.0])| [0.013102157639334477,0.08219716317632633]|         0.14775| 0.2026267184566039|
|(9,[0,2,7,8],[1.0,1.0,13.25,16.0])|[-0.024231767643273568,0.12517708677420586]|         0.07262| 0.1981638861389264|
|(9,[0,2,7,8],[1.0,1.0,13.25,28.0])| [-0.06903247798240324,0.17675299509166134]|         0.05281|0.19280848735771344|
|(9,[0,2,7,8],[1.0,1.0,13.25,31.0])|  [-0.08023265556718565,0.1896469721710252]|         0.06231| 0.1914696376624102|
|(9,[0,2,7,8],[1.0,1.0,13.25,34.0])| [-0.09143283315196807,0.20254094925038907]|         0.06231|0.19013078796710697|
|(9,[0,2,7,8],[1.0,1.0,13.25,37.0])| [-0.102633010736750

In [36]:
evaluator = RegressionEvaluator(labelCol='normalized_price', predictionCol='prediction', metricName='rmse')
rmse = evaluator.evaluate(test_with_pred)
print ("Root Mean Square of SVR model is %g"%rmse)

Root Mean Square of SVR model is 0.26827


In [37]:
# spark.stop